<a href="https://colab.research.google.com/github/samiha-mahin/A-Machine-Learning-Models-Repo/blob/main/Bagging_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### 🌳 What is Bagging?

**Bagging** stands for **Bootstrap Aggregating**.

It’s an **ensemble learning technique** that:

* Combines the predictions of **multiple models** (usually of the same type).
* Aims to **reduce variance** and **prevent overfitting**.
* Works especially well with **unstable models** (like decision trees).

---

### 🧠 How Does It Work?

1. **Bootstrapping (Sampling with replacement):**

   * From the original dataset, you create **multiple new datasets** by randomly selecting data points **with replacement**.
   * Each dataset is slightly different.

2. **Train models:**

   * You train a **separate model** on each of these datasets.
   * Typically, the base model is a **Decision Tree**.

3. **Aggregate the predictions:**

   * For **classification**: Take the **majority vote**.
   * For **regression**: Take the **average** of all outputs.

---

### 🍎 Real-Life Analogy

Imagine you're trying to decide whether a new fruit is an **apple or a tomato**.

* You ask **10 friends**, but give each of them a slightly different photo of the fruit (some brighter, some darker).
* Each friend gives you their answer.
* You **take the majority vote** to decide what the fruit is.

→ This is **bagging** in action — using **multiple weak opinions** to make a **strong, stable decision**.

---

### 📊 Proper Example: Predicting Loan Approval

Suppose you're building a model to predict if a person should get a loan.

#### Without Bagging:

* You train **one decision tree**.
* If the data changes a little, the tree might give **completely different results** (high variance).

#### With Bagging:

* You create 10 datasets from the original one (bootstrapped).
* Train 10 decision trees (one on each dataset).
* For a new applicant:

  * Each tree makes a prediction (Yes or No).
  * You take the **majority vote**.

✅ Result:

* **More stable**
* **Less likely to overfit**
* **Higher accuracy**

---

### 📦 Popular Bagging Example: **Random Forest**

* Random Forest = Bagging of **Decision Trees**
* It’s one of the most powerful and widely used ML algorithms.

---

### ✅ Benefits of Bagging

| Feature            | Benefit                                |
| ------------------ | -------------------------------------- |
| Reduces Variance   | Makes unstable models more stable      |
| Avoids Overfitting | Prevents one model from learning noise |
| Easy to Implement  | Especially with decision trees         |
| Improves Accuracy  | Better than a single model             |

